# 1. Data Portal Preprocessing

In [ ]:
try(library(tidyverse), silent=TRUE)
library(lubridate)
library(glue)
library(data.table)
library(tidyfast)
library("magrittr")
setwd("/")

In [ ]:
dataset_name = "name_of_your_dataset"
path = "/path/to/dir/with/decoded/file"
data_path = "/path/for/output"
dataset_path = glue("{data_path}/2_datasets_pre/{dataset_name}")

In [ ]:
list.files(path = "/data/project/uk_bb/cvd/data/ukb_downloads/updated_showcase_43098/ukb_data/records/")

## Load Athena Vocabulary

In [ ]:
vocab_dir = glue("{data_path}/athena_vocabulary_covid")
concept =fread(glue("{vocab_dir}/CONCEPT.csv"), sep='\t')

In [ ]:
unique(concept$vocabulary_id)

In [ ]:
relationship = fread(glue("{vocab_dir}/RELATIONSHIP.csv"), sep='\t')

In [ ]:
vocabulary =  fread(glue("{vocab_dir}/VOCABULARY.csv"), sep='\t')

In [ ]:
concept_relationship = fread(glue("{vocab_dir}/CONCEPT_RELATIONSHIP.csv"), sep='\t')

In [ ]:
## Diagnoses

## Hospital Episode Statistics

In [ ]:
hesin = fread("/data/project/uk_bb/cvd/data/ukb_downloads/updated_showcase_43098/ukb_data/records/hesin.txt")

In [ ]:
hesin_diag = fread("/data/project/uk_bb/cvd/data/ukb_downloads/updated_showcase_43098/ukb_data/records/hesin_diag.txt")

In [ ]:
hesin_critical = fread("/data/project/uk_bb/cvd/data/ukb_downloads/updated_showcase_43098/ukb_data/records/hesin_critical.txt")

In [ ]:
hesin_psych = fread("/data/project/uk_bb/cvd/data/ukb_downloads/updated_showcase_43098/ukb_data/records/hesin_psych.txt")

In [ ]:
hesin_delivery = fread("/data/project/uk_bb/cvd/data/ukb_downloads/updated_showcase_43098/ukb_data/records/hesin_delivery.txt")
hesin_maternity = fread("/data/project/uk_bb/cvd/data/ukb_downloads/updated_showcase_43098/ukb_data/records/hesin_maternity.txt")

### Diagnoses - ICD10

In [ ]:
## icd9 to icd10 mapping
icd9to10_df = fread("/data/analysis/ag-reils/steinfej/code/umbrella/pre/ukbb/mapping/codings/coding1836.tsv")
icd9to10_mapping = split(icd9to10_df$meaning, icd9to10_df$coding)
hesin_diag_icd9 = hesin_diag %>% filter(diag_icd9!="") %>% rowwise() %>% mutate(diag_icd10 = list(icd9to10_mapping[[diag_icd9]])) %>% drop_na(diag_icd10)
hesin_diag = rbind(hesin_diag %>% filter(diag_icd9=="") %>% mutate(origin="hes_icd10"), hesin_diag_icd9  %>% mutate(origin="hes_icd9"))

In [ ]:
hes_join = hesin[hesin_diag, on=c("eid", "ins_index")]
hes_join = hes_join[, c("eid", "origin","ins_index", "arr_index", "level", "epistart", "diag_icd10")][order(eid, ins_index, arr_index),]

In [ ]:
hes_join_date = hes_join %>% rename(date="epistart") %>% mutate(date = ymd(as.Date(fast_strptime(date, "%d/%m/%Y"))))

In [ ]:
hes_diagnoses = hes_join_date %>% drop_na(date) %>% rename(code = "diag_icd10") %>% mutate(instance=ins_index) %>% group_by(eid) %>% mutate(n = arr_index)

In [ ]:
hes_diagnoses = hes_diagnoses %>% mutate(meaning=str_sub(code, 1, 3)) %>% select(c(eid, origin, instance, n, level, code, meaning, date))

In [ ]:
nrow(hes_diagnoses)
head(hes_diagnoses %>% arrange(desc(date)))

In [ ]:
arrow::write_feather(hes_diagnoses, glue("{path}/codes_hes_diagnoses_210120.feather"))

### Procedures - Snomed CT

In [ ]:
hesin_oper = fread("/data/project/uk_bb/cvd/data/ukb_downloads/updated_showcase_43098/ukb_data/records/hesin_oper.txt")

In [ ]:
hesin_oper[hesin_oper == ""] <- NA


In [ ]:
hesin_oper_pre = hesin_oper %>% rename(date="opdate", code="oper4") %>% 
    mutate(date = ymd(as.Date(fast_strptime(date, "%d/%m/%Y"))))  %>%
    mutate(origin="hes_opcs4", instance=ins_index) %>% group_by(eid) %>% mutate(n = arr_index) %>% select(eid, origin, instance, n, level, code, date)

In [ ]:
concept_ids_opcs4 = concept %>% filter(vocabulary_id == "OPCS4") %>% mutate(concept_code = str_replace(concept_code, "\\.", ""))
concept_ids_snomed = concept %>% filter(vocabulary_id == "SNOMED" & domain_id=="Procedure") 

# check necessary opcs4 concept ids
concept_ids = concept_ids_opcs4 %>% mutate(concept_id_1 = concept_id)

cr_filtered = concept_relationship %>% filter(concept_id_1 %in% concept_ids_opcs4$concept_id) %>% filter(concept_id_2 %in% concept_ids_snomed$concept_id) %>% arrange(concept_id_1)

In [ ]:
mapping_opcs4_snomed = concept_ids_opcs4 %>% 
    left_join(cr_filtered %>% select(concept_id_1, concept_id_2), by=c("concept_id"="concept_id_1")) %>% 
    left_join(concept_ids_snomed %>% select(concept_id, concept_code, concept_name), by=c("concept_id_2"="concept_id")) %>% 
    mutate(code = concept_code.x, meaning=concept_code.y, name=concept_name.y)

In [ ]:
hes_procedures = hesin_oper_pre %>% left_join(mapping_opcs4_snomed %>% select(code, meaning, name), by="code") %>% select(eid, origin, instance, n, level, date, code, meaning, name)

In [ ]:
arrow::write_feather(hes_procedures, glue("{path}/codes_hes_procedures_210119.feather"))

## Mortality Records - ICD10

In [ ]:
death = fread("/data/project/uk_bb/cvd/data/ukb_downloads/updated_showcase_43098/ukb_data/records/death.txt")
death_cause = fread("/data/project/uk_bb/cvd/data/ukb_downloads/updated_showcase_43098/ukb_data/records/death_cause.txt")

In [ ]:
death_join = death[death_cause, on=c("eid", "ins_index")]
death_join = death_join[, c("eid", "ins_index", "arr_index", "level", "date_of_death", "cause_icd10")][order(eid, ins_index, arr_index),]

In [ ]:
death_join_date = death_join %>% rename(date="date_of_death") %>% rename(code = "cause_icd10") %>% mutate(date = ymd(as.Date(fast_strptime(date, "%d/%m/%Y"))))

In [ ]:
codes_death = death_join_date  %>% mutate(instance=0) %>% mutate(origin="death_records") %>% group_by(eid) %>% mutate(n=row_number())
codes_death = codes_death %>% mutate(meaning=str_sub(code, 1, 3)) %>% select(c(eid, origin, instance, n, level, code, meaning, date))

In [ ]:
arrow::write_feather(codes_death, glue("{path}/codes_death_records_210115.feather"))

## GP Records

In [ ]:
gp_registrations = fread("/data/project/uk_bb/cvd/data/ukb_downloads/updated_showcase_43098/ukb_data/records/gp_registrations.txt")

In [ ]:
gp_clinical = fread("/data/project/uk_bb/cvd/data/ukb_downloads/updated_showcase_43098/ukb_data/records/gp_clinical.txt")

In [ ]:
gp_clinical[gp_clinical == ""] <- NA

In [ ]:
gp_clinical = gp_clinical %>% rename(date="event_dt") %>% mutate(date = ymd(as.Date(fast_strptime(date, "%d/%m/%Y"))))

In [ ]:
# clean_dates
# These data are provided in a form which is as close as possible to how they were issued from their source supplier, in order to avoid potential systematic error or bias by attempting to ‘clean’ them by
# removing or altering invalid or erroneous information. However, to protect individuals, alterations have been made to dates in relation to participant date of birth as follows:

# - where clinical event or prescription date precedes participant date of birth it has been altered to 01/01/1901.
# - Where the date matches participant date of birth it has been altered to 02/02/1902.
# - Where the date follows participant date of birth but is in the year of their birth it has been altered to 03/03/1903.
# - Where the date was in the future this has been changed to 07/07/2037 as these are likely to have been entered as a place-holder or other system default.

In [ ]:
gp_clinical = gp_clinical %>% filter(date!="2037-07-07")

### Diagnoses - ICD10

In [ ]:
readv2_icd10 = head(fread("/data/analysis/ag-reils/steinfej/code/umbrella/pre/ukbb/mapping/gp_codings/read_v2_icd10.csv"), -3) %>% rename(read_2="read_code", code ="icd10_code") %>% select(read_2, code)
readv3_icd10 = head(fread("/data/analysis/ag-reils/steinfej/code/umbrella/pre/ukbb/mapping/gp_codings/read_ctv3_icd10.csv"), -3)%>% rename(read_3="read_code", code="icd10_code") %>% select(read_3, code)

In [ ]:
gp_diagnoses_pre = gp_clinical %>% filter(read_2 %in% readv2_icd10$read_2 | read_3 %in% readv3_icd10$read_3)
gp_diagnoses_readv2 = gp_diagnoses_pre %>% filter(!is.na(read_2)) %>% left_join(readv2_icd10, on="read_2") %>% drop_na(code) %>% mutate(origin="gp_read2") %>% select(eid, origin, code, date)
gp_diagnoses_readv3 = gp_diagnoses_pre %>% filter(!is.na(read_3)) %>% left_join(readv3_icd10, on="read_3") %>% drop_na(code) %>% mutate(origin="gp_read3") %>% select(eid, origin, code, date)
gp_diagnoses_raw = rbind(gp_diagnoses_readv2, gp_diagnoses_readv3)

In [ ]:
gp_diagnoses = gp_diagnoses_raw %>% mutate(instance=0, level=NA) %>% distinct() %>% group_by(eid) %>% mutate(n = row_number()) %>% mutate(meaning=str_sub(code, 1, 3)) %>% select(c(eid, origin, instance, n, level, code, meaning, date))

In [ ]:
arrow::write_feather(gp_diagnoses, glue("{path}/codes_gp_diagnoses_210119.feather"))

### Procedures - Snomed CT

In [ ]:
readv2_opcs4 = head(fread("/data/analysis/ag-reils/steinfej/code/umbrella/pre/ukbb/mapping/gp_codings/read_v2_opcs4.csv"), -3) %>% rename(read_2="read_code", code ="opcs_4.2_code") %>% select(read_2, code)
readv3_opcs4 = head(fread("/data/analysis/ag-reils/steinfej/code/umbrella/pre/ukbb/mapping/gp_codings/read_ctv3_opcs4.csv"), -3)%>% rename(read_3="read_code", code="opcs4_code") %>% select(read_3, code)

In [ ]:
gp_procedures_pre = gp_clinical %>% filter(read_2 %in% readv2_opcs4$read_2 | read_3 %in% readv3_opcs4$read_3)

In [ ]:
gp_procedures_readv2 = gp_procedures_pre %>% filter(!is.na(read_2)) %>% left_join(readv2_opcs4, on="read_2") %>% drop_na(code) %>% mutate(origin="gp_read2") %>% select(eid, origin, code, date)
gp_procedures_readv3 = gp_procedures_pre %>% filter(!is.na(read_3)) %>% left_join(readv3_opcs4, on="read_3") %>% drop_na(code) %>% mutate(origin="gp_read3") %>% select(eid, origin, code, date)

In [ ]:
gp_procedures_raw = rbind(gp_procedures_readv2, gp_procedures_readv3) %>% mutate(instance=0, level=NA) %>% distinct() %>% group_by(eid) %>% mutate(n = row_number()) 

In [ ]:
# opcs4 to snomed mapping

concept_ids_opcs4 = concept %>% filter(vocabulary_id == "OPCS4") %>% mutate(concept_code = str_replace(concept_code, "\\.", ""))
concept_ids_snomed = concept %>% filter(vocabulary_id == "SNOMED" & domain_id=="Procedure") 

# check necessary opcs4 concept ids
concept_ids = concept_ids_opcs4 %>% mutate(concept_id_1 = concept_id)
cr_filtered = concept_relationship %>% filter(concept_id_1 %in% concept_ids_opcs4$concept_id) %>% filter(concept_id_2 %in% concept_ids_snomed$concept_id) %>% arrange(concept_id_1)

mapping_opcs4_snomed = concept_ids_opcs4 %>% 
    left_join(cr_filtered %>% select(concept_id_1, concept_id_2), by=c("concept_id"="concept_id_1")) %>% 
    left_join(concept_ids_snomed %>% select(concept_id, concept_code, concept_name), by=c("concept_id_2"="concept_id")) %>% 
    mutate(code = concept_code.x, meaning=concept_code.y, name=concept_name.y)

In [ ]:
gp_procedures = gp_procedures_raw %>% left_join(mapping_opcs4_snomed %>% select(code, meaning, name), by="code") %>% select(eid, origin, instance, n, level, date, code, meaning, name) %>% arrange(eid, date)

In [ ]:
arrow::write_feather(gp_procedures, glue("{path}/codes_gp_procedures_210119.feather"))

### Measurements - Snomed CT

In [ ]:
readv2_readv3 = head(fread("/data/analysis/ag-reils/steinfej/code/umbrella/pre/ukbb/mapping/gp_codings/read_v2_read_ctv3.csv"), -3) %>% rename(read_2="READV2_CODE", code ="READV3_CODE", name ="TERMV3_DESC") %>% select(read_2, code)

In [ ]:
gp_meas = gp_clinical %>% filter(!is.na(value1)) %>% distinct()

In [ ]:
gp_meas_readv2 = gp_meas %>% filter(!is.na(read_2)) %>% left_join(readv2_readv3, by="read_2")

In [ ]:
gp_meas_readv3 = gp_meas %>% filter(!is.na(read_3)) %>% mutate(code=read_3)

In [ ]:
gp_meas_all = rbind(gp_meas_readv2, gp_meas_readv3) %>% distinct() %>% group_by(eid) 

In [ ]:
readv3_lkp = head(fread("/data/analysis/ag-reils/steinfej/code/umbrella/pre/ukbb/mapping/gp_codings/read_ctv3_lkp.csv"), -3)%>% rename(code="read_code", name ="term_description") %>% select(code, name)
readv3_sct = fread("/data/analysis/ag-reils/steinfej/code/umbrella/pre/ukbb/mapping/gp_codings/CTV3SCTMAP.csv")%>% rename(SCUI="V1", STUI="V2", TCUI="V3", TTUI="V4")%>% rename(code="SCUI", meaning="TCUI") %>% select(code, meaning)
#readct_sct = fread("/data/analysis/ag-reils/steinfej/code/umbrella/pre/ukbb/mapping/gp_codings/RCTSCTMAP.csv")%>% rename(SCUI="V1", STUI="V2", TCUI="V3", TTUI="V4")#%>% rename(code="read_code", name ="term_description") %>% select(code, name)#

In [ ]:
gp_meas = gp_meas_all %>% left_join(readv3_lkp, by="code")

In [ ]:
concept_ids_snomed = concept %>% filter(vocabulary_id == "SNOMED") %>% rename(name="concept_name", meaning="concept_code") %>% select(meaning, name)

In [ ]:
gp_meas_uncleaned = gp_meas_all %>% left_join(readv3_sct, by="code") %>% left_join(concept_ids_snomed, by="meaning") %>% distinct()

In [ ]:
gp_meas_cleaned_1 = gp_meas_uncleaned %>% select(eid, date, code, value1, value2, value3, meaning, name) %>% distinct() %>% filter(value1!=0)

In [ ]:
gp_meas_cleaned_2 = gp_meas_cleaned_1 %>% ungroup() %>% filter(!is.na(meaning))

In [ ]:
double_df = gp_meas_cleaned_2 %>% filter(!is.na(as.numeric(value1)) & !is.na(as.numeric(value2))) 

In [ ]:
# clean blood pressure and map to systolic and diastolic
bp_double_mapped = double_df %>% filter(name %in% c('O/E - blood pressure reading', 'O/E - BP reading normal', 'O/E - BP reading raised',
                'O/E - BP borderline raised', 'O/E - Systolic BP reading', 'O/E - Diastolic BP reading', 'Sitting blood pressure', "Average home systolic blood pressure",
                'Standing blood pressure','24 hr blood pressure monitoring')) %>% 
     #filter(name %in% c('O/E - Systolic BP reading', 'O/E - Diastolic BP reading', "Average home systolic blood pressure")) %>%
    filter(as.numeric(value1)>0) %>% 
    mutate(value_high = pmax(as.numeric(value1), as.numeric(value2)), value_low = pmin(as.numeric(value1), as.numeric(value2))) %>% 
    filter(value_high>40 & value_low>20 & value_high<400 & value_low<300)  %>% rename("163030003" = "value_high", "163031004" = "value_low") %>% 
    select(-c(meaning, name)) %>% pivot_longer(c("163030003", "163031004"), names_to="meaning", values_to="value") %>% left_join(concept_ids_snomed, by="meaning") %>% distinct() %>% arrange(eid) %>%
    select(eid, date, code, value1, value2, value3, meaning, name, value)

In [ ]:
gp_meas_single = gp_meas_cleaned_2 %>% filter(is.na(as.numeric(value1)) | is.na(as.numeric(value2))) %>%
    mutate(value=case_when(!is.na(as.numeric(value1)) ~ as.numeric(value1), is.na(as.numeric(value1)) ~ as.numeric(value2))) %>% filter(!is.na(value))

In [ ]:
gp_meas_cleaned_3 = rbind(gp_meas_single, bp_double_mapped) %>% distinct() %>% arrange(eid, date)

In [ ]:
arrow::write_feather(gp_meas_cleaned_3, glue("{path}/codes_gp_measurements_210120.feather"))

### Prescriptions - RXNorm

In [ ]:
gp_scripts = fread("/data/project/uk_bb/cvd/data/ukb_downloads/updated_showcase_43098/ukb_data/records/gp_scripts.txt")

In [ ]:
gp_scripts[gp_scripts == ""] <- NA

In [ ]:
gp_scripts = gp_scripts %>% mutate(date = ymd(as.Date(fast_strptime(issue_date, "%d/%m/%Y"))))

In [ ]:
gp_scripts_names_available = gp_scripts %>% filter(!is.na(drug_name))

In [ ]:
gp_scripts_read_available = gp_scripts %>% filter(is.na(drug_name))

In [ ]:
drug_names = (gp_scripts_names_available %>% count(drug_name, sort=TRUE))$drug_name

In [ ]:
library(jsonlite)
write_json(drug_names, glue("{path}/drug_names.json"))

In [ ]:
name_umls_link = arrow::read_feather(glue("{path}/drug_names_umls_linked.feather"))

In [ ]:
drugs_rxnorm = arrow::read_feather(glue("{path}/drug_names_umls_linked_rxnorm.feather"))

In [ ]:
rx_mapping = concept %>% filter(vocabulary_id %in% c('RxNorm','RxNorm Extension')) %>% select(concept_code, concept_name) %>% rename(rx_code ="concept_code", name="concept_name")

In [ ]:
rx_norm_mapping_table = drugs_rxnorm %>% select(drug_name, rx_code) %>% filter(rx_code != "") %>% distinct() %>% left_join(rx_mapping, on="rx_code")

In [ ]:
gp_scripts_rxnorm = gp_scripts_names_available %>% left_join(rx_norm_mapping_table, on="drug_name") %>% select(eid, date, drug_name, rx_code, name) %>% distinct()

In [ ]:
arrow::write_feather(gp_scripts_rxnorm, glue("{path}/codes_gp_prescription_scispacy.feather"))